In [262]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [263]:
dvach = open('Рабочий стол/2ch_corpus.txt').read()[:100000]
news = open('Рабочий стол/lenta.txt').read()[:100000]

In [264]:
from nltk.tokenize import sent_tokenize

In [265]:
sentences_dvach = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [266]:
unigrams_dvach = Counter()
trigrams_dvach = Counter()
bigrams_dvach = Counter()
for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(bigrammer(sentence,2))
    trigrams_dvach.update(ngrammer(sentence,3))


unigrams_news = Counter()
trigrams_news = Counter()
bigrams_news = Counter()
for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence,2))
    trigrams_news.update(ngrammer(sentence,3))


In [267]:
# это триграммы
trigrams_news.most_common(10)

[(('<start>', '<start>', 'в'), 54),
 (('<start>', '<start>', 'по'), 48),
 (('<start>', '<start>', 'как'), 32),
 (('<start>', '<start>', 'на'), 14),
 (('<start>', '<start>', 'об'), 14),
 (('<start>', 'об', 'этом'), 13),
 (('<start>', 'по', 'данным'), 12),
 (('bank', 'of', 'new'), 11),
 (('of', 'new', 'york'), 11),
 (('<start>', '<start>', 'это'), 10)]

In [268]:
#  Массив из нулей указанных размеров
matrix_news = np.zeros((len(bigrams_news), 
                   len(unigrams_news)))
# униграммы
id2word_news = list(unigrams_news)
#print(id2word_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}
#print(word2id_news)
# биграммы
id2bigram_news = list(bigrams_news)
#print(id2bigram_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}
#print(bigram2id_news)
    

# вероятность можно получить поделив количесво вхождений триграмма на количество вхождений биграмма
for ngram in trigrams_news:
    word1, word2, word3 = ngram
    bigramm = (word1,word2)
    #int(bigram2id_news[bigramm])
    matrix_news[bigram2id_news[bigramm]][word2id_news[word3]] = (trigrams_news[ngram]/bigrams_news[bigramm])

In [269]:
#  Массив из нулей указанных размеров
matrix_dvach = np.zeros((len(bigrams_dvach), 
                   len(unigrams_dvach)))
# униграммы
id2word_dvach = list(unigrams_dvach)
#print(id2word_news)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
#print(word2id_news)
# биграммы
id2bigram_dvach = list(bigrams_dvach)
#print(id2bigram_news)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}
#print(bigram2id_news)
    

# вероятность можно получить поделив количесво вхождений триграмма на количество вхождений биграмма
for ngram in trigrams_dvach:
    word1, word2, word3 = ngram
    bigramm = (word1,word2)
    #print(bigram2id_news[bigramm])
    matrix_dvach[bigram2id_dvach[bigramm]][word2id_dvach[word3]] = (trigrams_dvach[ngram]/bigrams_dvach[bigramm])
print(matrix_dvach)

[[0.00000000e+00 6.89655172e-04 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [364]:
def generate(matrix, id2word, bigram2id, n=100, start = ('<start>','<start>')):
    # у меня есть список униграм и словарь из биграм; 
    # я пытаюсь составить текст длинны n, где маркер начала этого текста start = ('<start>','<start>')
    # этот маркер будет типа первая триграмма текста, current_bigramm_idx = что тут я не поняла[start], 
    # но по идее если это такая триграмма, ее надо в триграммах искать
    # маркер начала текста типа старт старт то есть первое слово и впереди два тега старт
    # пока слов не станет ровно n, я заполняю матрицу словами
    # слова выбираются функцией np.random.choice
    # я добавляю туда параметр p=matrix[current_bigramm_idx], чтобы найти слово по его вероятности 
    # если нашелся тег конца, надо перейти на начало
    # перезиписать индекс на новый
    # print(current_idx) 
    #ну вот вы взяли получили первый индекс, перевели его в слово, добавли к тексту, 
    #теперь вам нужно взять второй старт и это первое слово, из маппинга
    #для биграммов достать индекс (второго старта, первого слова) и положить этот индекс в current_bigramm_idx
    
    text = []
    current_bigramm_idx = bigram2id[start]
    
    for i in range(n):
        # тут мы получаем первый индекс рандомного слова
        chosen = np.random.choice(matrix.shape[1],p=matrix[current_bigramm_idx])
        #print(chosen)
        
        #print(current_word2)
        # находим это слово в списке по его индексу и добавляем в текст
        text.append(id2word[chosen])
        current_word2 = ''
        # если это слово имеет вид '<end>'
        if id2word[chosen] == '<end>':
            # то мы меняем индекс
            #print(id2word[chosen])
            chosen = bigram2id[start]
            #print(chosen,'chosen')
        current_bigramm_idx = bigram2id[(current_word2, id2word[chosen])]
        #print(chosen)
    
    return ' '.join(text)

In [365]:
print(generate(matrix_dvach, id2word_dvach, bigram2id_dvach).replace('<end>', '\n'))

KeyError: ('', 'хорошо')

In [356]:
print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

KeyError: ('<start>', 'августа')